# Import and Setup

In [225]:
# Import necessary libraries
import os
import json
import random
from dotenv import load_dotenv
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

In [226]:
# Load API keys from .env file
# Replace this with the path to where your .env file is, which should contain openAI API keys as well as other necessary environment variables
# Env file should contain something like this:
# OPENAI_API_KEY=your openai api key
# OPENAI_ORG_ID=your org id
# LANGCHAIN_TRACING_V2='true'
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY=your langchain api key
# LANGCHAIN_PROJECT=the project name you want to use
load_dotenv("../.env") 

# Set environment variables for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [227]:
# Set up the OpenAI model
model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')

In [228]:
### Specify patient transcript file to read in
transcripts_version = 1.0

# System Prompts

In [229]:
system_message_summarize_detailed = """

You are a medical assistant tasked with reviewing a transcript of a conversation between a patient and their doctor. You will be provided a transcript. The doctor has asked you to write up a summary of the transcript in the format outlined below. Include section headings and use bullet points. Add context to symptoms where appropriate, but be brief. List specific medications by name under the appropriate medication category. Do not add any information that is not present in the transcript.

Patient Introduction:
    - Write a one sentence summary including the patient's name and their primary symptom or chief complaint

Current Symptoms (Note: Determine which symptoms from the list below are present and not present. Only list those present in the transcript.):
     - Dyspnea
     - Paroxysmal Nocturnal Dyspnea (PND)
     - Orthopnea
     - Edema
     - Nocturnal Cough
     - Chest Pain
     - Fatigue and Mental Status

Vital Signs
     - Temperature (°F): 
     - Heart Rate (BPM): 
     - Respiratory Rate (breaths per minute):
     - Oxygen Saturation: 
     - Blood Pressure: 
     - Weight (in lbs): 

Current Medications:
     - List the medications the patient is taking

Summary:
     - Write a one or two sentence summary of the conversation. 
     - Avoid making any diagnoses; just report facts. 
     - Each new sentence should be a new bullet point.

"""

# Load Transcripts Data

In [230]:
## Load patient data

# Specify the path to your JSON file
json_file_path = f"../data/patients/patients_{transcripts_version}_with_transcripts.json"

# Open and read the JSON file
with open(json_file_path, 'r') as json_file:
    patients = json.load(json_file)

In [231]:
random_key = random.choice(list(patients.keys()))

In [242]:
patient_transcript = patients[random_key]['chat_transcript']

#for line in patient_transcript:
#    print(line)

In [233]:
patient_transcript_string = ""

for line in patient_transcript:
    patient_transcript_string = patient_transcript_string + line

# Generate Example Summary

In [234]:
parser = StrOutputParser()

In [235]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_message_summarize_detailed), ("user", patient_transcript)]
)

In [236]:
chain = prompt_template | model | parser

In [237]:
result = chain.invoke({"system": system_message_summarize_detailed, "text": patient_transcript})

In [238]:
print(result)

Patient Introduction:
- Gregory Reynolds is experiencing shortness of breath, especially with activity.

Current Symptoms:
- Dyspnea (shortness of breath with activity)
- Fatigue
- Trouble sleeping
- Feeling down
- Lightheadedness upon standing

Vital Signs
- Temperature (°F): 98.4
- Heart Rate (BPM): 106
- Respiratory Rate (breaths per minute): 18
- Oxygen Saturation: 99.0%
- Blood Pressure: 114/57

Current Medications:
- Carvedilol (Beta-Blocker)
- Nitroglycerin
- Aspirin
- Metolazone
- Dronedarone

Summary:
- Gregory Reynolds is experiencing dyspnea mainly with activity, fatigue, trouble sleeping, feeling down, and lightheadedness upon standing.
- He is currently taking Carvedilol, Nitroglycerin, Aspirin, Metolazone, and Dronedarone for his heart condition.
- Vital signs including temperature, heart rate, respiratory rate, oxygen saturation, and blood pressure were within normal ranges during the visit.


## Generate Summaries for All Patients in Transcripts File

In [239]:
# Specify the path to your new summaries JSON file
summary_json_file_path = f"../data/patients/patients_{transcripts_version}_summaries.json"

In [240]:
summaries = {}

for p_id in patients:
    summary = {}

    summary['id'] = p_id

    parser = StrOutputParser()
    prompt_template = ChatPromptTemplate.from_messages(
        [("system", system_message_summarize_detailed), ("user", patients[p_id]['chat_transcript'])]
    )
    chain = prompt_template | model | parser

    summary['summary'] = chain.invoke({"system": system_message_summarize_detailed, "text": patients[p_id]['chat_transcript']})

    summaries[str(p_id)] = summary

In [241]:
summaries_json_file_name = f"../data/patients/patients_{transcripts_version}_summaries.json"

with open(summaries_json_file_name, 'w') as json_file:
    json.dump(summaries, json_file)